In [1]:
import urllib
from urllib2 import urlopen
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bs4 import BeautifulSoup
from collections import defaultdict
from bson.objectid import ObjectId
%matplotlib inline

db_client = MongoClient()
data = db_client['shopify']['forum']
data2 = db_client['shopify_backup']
users = db_client['shopify']['users']
post_db = db_client['shopify']['posts']
shopify_post_db = db_client['shopify']['shopify_posts']

In [2]:
base_url = 'https://ecommerce.shopify.com/'
forums_url = 'forums'

<div style="text-align:center;padding:10px;"><h1>Step 3: Scrape User and Topic Information</h1></div>

In [5]:
for category in [data.find({'category':'category-shopify-discussion'})[0]]:
    
    print '-----> CATEGORY:', category['category']
    for sub_cat_key in category["sub_cats"]:
        print sub_cat_key
        
        if sub_cat_key == 'category-shopify-discussion':
            
            print 'SUBCATEGORY: {sub_cat}'.format(sub_cat=sub_cat_key)


            sub_cat = category['sub_cats'][sub_cat_key]
            posts = []


            # Run through each thread
            for idx, url in enumerate(sub_cat['topic_urls']):

                if idx > 12360:
                    try:
                        topic_page = BeautifulSoup(urlopen(url))
                        thread = []

                        # Process original post
                        user_id, text, date = process_post(topic_page.find('div',{'class':'original-post'}))
                        shopify_post_db.insert_one({'user_id':user_id, 'date':date, 'text':text})

                        # Process each reply
                        for reply in topic_page.find_all('div',{'itemtype':'http://schema.org/UserComments'}):
                            user_id, text, date = process_post(reply)
                            shopify_post_db.insert_one({'user_id':user_id, 'date':date, 'text':text})


                        # Status Update
                        if (idx+1) % 5 == 0:
                            print 'PROCESSED {idx}/{total} posts'.format(idx=(idx+1),total=len(sub_cat['topic_urls']))

                    except Exception as e:
                        print 'ERROR IN SUBCATEGORY: ', str(e)

-----> CATEGORY: category-shopify-discussion
category-shopify-point-of-sale
category-shopify-job-board
category-scripts
category-shopify-apps
category-shopify-mobile
category-api-announcements
category-shopify-apis-and-technology
category-ecommerce-design
category-shopify-discussion
SUBCATEGORY: category-shopify-discussion


/Users/brandongarcia/miniconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/brandongarcia/miniconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


PROCESSED 12365/15380 posts
PROCESSED 12370/15380 posts
PROCESSED 12375/15380 posts
PROCESSED 12380/15380 posts
PROCESSED 12385/15380 posts
PROCESSED 12390/15380 posts
PROCESSED 12395/15380 posts
PROCESSED 12400/15380 posts
PROCESSED 12405/15380 posts
PROCESSED 12410/15380 posts
PROCESSED 12415/15380 posts
PROCESSED 12420/15380 posts
PROCESSED 12425/15380 posts
PROCESSED 12430/15380 posts
PROCESSED 12435/15380 posts
PROCESSED 12440/15380 posts
PROCESSED 12445/15380 posts
PROCESSED 12450/15380 posts
PROCESSED 12455/15380 posts
PROCESSED 12460/15380 posts
PROCESSED 12465/15380 posts
PROCESSED 12470/15380 posts
PROCESSED 12475/15380 posts
PROCESSED 12480/15380 posts
PROCESSED 12485/15380 posts
PROCESSED 12490/15380 posts
PROCESSED 12495/15380 posts
PROCESSED 12500/15380 posts
PROCESSED 12505/15380 posts
PROCESSED 12510/15380 posts
PROCESSED 12515/15380 posts
PROCESSED 12520/15380 posts
PROCESSED 12525/15380 posts
PROCESSED 12530/15380 posts
PROCESSED 12535/15380 posts
PROCESSED 12540/1538

In [3]:
def process_post(post):
    
    user = post.find('div',{'class':'author'})
    user_id = int(user.find('a',{'class':'username'}).get('href').split('/')[-1])
    
    # Add user to db if does not exist 
    if users.find({'id': user_id}).count() < 1:
        name = user.find('a',{'class':'username'}).text.strip()
        title = user.find_all('em')[0].text.strip().lower()
        site = user.find_all('em')[-1].text.strip().lower()
        posts = int(user.find('div',{'class':'stats'}).find('a').text.strip().lower())
        users.insert_one({'id':user_id,'name':name,'title':title,'site':site, 'posts':posts })
        
    # Extract date
    date = get_date(post)
    
    # Extract text
    text = ' '.join([txt.text.lower() for txt in post.find('div',{'itemprop':'description'}).find_all('p')])
    
    return user_id, text, date
    
    


In [4]:
import re
import datetime as dt
months = ['january','february','march','april','may','june','july','august','september','october','november','december']

def get_date(post):
    
    regexes = ['(\d+) day','(\d+) month', '(\w+) (\d+), (20\d+)']
    raw_date = post.find('div',{'class':'date'}).text.strip()
    date = None
    
    for idx, reg in enumerate(regexes):
        matches = re.search(reg, raw_date)
        if matches:
            if idx == 0:
                date = dt.datetime.now() - dt.timedelta(days=int(matches.group(1)))
            elif idx == 1:
                date = dt.datetime.now() - dt.timedelta(days=int(matches.group(1))*30)
            else:
                date = dt.datetime(int(matches.group(3)), int(months.index(matches.group(1).lower())) + 1, int(matches.group(2)))

    return date.strftime('%Y-%m-%d') if date else 'WHY'